In [2]:
from header import *
plt.rcParams.update({'font.size': 14})

In [3]:
df_seeds = read_astarix_performance('results/ecoli_illumina250_astar-seeds.tsv')
df_base = read_astarix_performance('results/ecoli_illumina250_astar-prefix.tsv')

df_seeds = read_astarix_performance('results/MHC1-astarix-prefix/alignments.tsv')
df_base = read_astarix_performance('results/MHC1-astarix-seeds/alignments.tsv')

display(df_seeds.head())

,ref,refsize,algo,operation,memory,len,read,spell,cost,starts,pushed,popped,repeat_rate,t(map),t(astar),unique_best,pushed+popped,generated_errors,explored_states,dist
readname,,,,,,,,,,,,,,,,,,,,
simulated.49,graphs/pasgal-MHC1.gfa,11611462,astar-prefix,align,0.0,100,TACCAGTACCATGCTGTTTTGGTTACTGTAGCCTTGAAGTATAGTT...,TACCAGTACCATGCTGTTTTGGTTACTGTAGCCTTGAAGTATAGTT...,0,-1,4.09,0.66,0.0,0.040621,0.015261,1,4.75,-1,409.0,0
simulated.17,graphs/pasgal-MHC1.gfa,11611462,astar-prefix,align,0.0,100,CCTCCACATGCAGGTGCCCGTCGATCCCAGCATGACTTTGCTCTCC...,CCTCCACATGCAGGTGCCcGTCGATCCCAGCATGACTTTGCTCTCC...,1,-1,9.76,0.97,0.0,0.096706,0.051567,1,10.73,-1,976.0,1
simulated.18,graphs/pasgal-MHC1.gfa,11611462,astar-prefix,align,0.0,100,ATGCAGTGGCACAATCTCTGTTCACTGCAGCCTCGGCCTCCCAGGC...,ATGCAGTGGCACAATCTCTGTTCACTGCAGCCTCGGCCTCCCAGGC...,0,-1,3.82,0.27,0.0,0.023336,0.002372,1,4.09,-1,382.0,0
simulated.19,graphs/pasgal-MHC1.gfa,11611462,astar-prefix,align,0.0,100,TGTACCCTAAAACTTAAAGTATAATAATAATAATAATAATAATAAA...,TGTACCCTAAAACTTAAAGTATAATAATAATAATAATAATAATAAA...,0,-1,2.56,0.39,0.0,0.035837,0.014740,1,2.95,-1,256.0,0
simulated.50,graphs/pasgal-MHC1.gfa,11611462,astar-prefix,align,0.0,100,GAGAACACAGGTAATAGGGAGAATGTAGGTAAAAAGAATCTCAACC...,GAGAACACAGGTAATAGGGAGAATGTAGGTAAAAAGAATcTCAACC...,1,-1,9.96,1.09,0.0,0.099725,0.054174,1,11.05,-1,996.0,1


In [4]:
def cross_validation(seeds, base):
    correct = True
    for index, row in df_seeds.iterrows():
        #display(row)
        a = df_base[df_base['read'] == row['read']]
        assert(len(a) == 1)
        r = a.iloc[0]
        if row.cost != r.cost:
            print("Mismatching costs: ", row.cost, " != ", r.cost)
            print(row)
            print(r)
            correct = False
    print('Checked ' + str(len(df_seeds)) + ' alignments.')
    return correct

In [5]:
cross_validation(df_seeds, df_base)

Checked 100 alignments.


True